### This is the Primary Notebook for the Project
If you have not run the preprocessing notebook, please do so before running this notebook. This notebook will perform the following tasks:
- Load the preprocessed data
- Define the Mimic3Dataset class to create a PyTorch Dataset
- Define the DST model

 #### Mount Google Drive and change directory to the project folder

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/dl4h_project/DynST/
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Mimic3Dataset Class
This will be called later to create the dataset for training and testing the model. It will read the csv file created by the Mimic3Pipeline class, and create the dataset for training and testing the model.
Also provided are the functions to pad and collate the data for use by the model.

In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

In [ ]:
class Mimic3Dataset(Dataset):
    def __init__(self, work_dir, seed, intervention=None):
        fdir = f"{work_dir}/data/preprocessed_{seed}"
        self.f = {}
        for fname in os.listdir(fdir):
            if fname.endswith(".npy"):
                self.f[fname[:-4]] = np.load(
                    f"{fdir}/{fname}", allow_pickle=True
                    )
        self.ix = self.f["patient_index"]
        self.code_lookup = np.insert(self.f["code_lookup"], 0, "pad")
        self.codes = self.f["codes"] + 1
        self.n_codes = len(self.code_lookup)
        self.n_vitals = self.f["vitals"].shape[1]
        self.n_demog = self.f["demog"].shape[1]
        self.pad_value = - 100
        # if supplied, represents treatment (True) or control (False)
        self.intervention = intervention



    def __len__(self):
        return len(self.f["treatment"])

    def __getitem__(self, index):
        item = {}
        j = self.ix[index]
        if self.intervention is None:
            item["treatment"] = self.f["treatment"][index]
        else:
            item["treatment"] = int(self.intervention)
        item["demog"] = self.f["demog"][index]
        item["codes"] = torch.tensor(
            self.pad_bincount(self.f["codes"][self.f["code_index"] == j])
        )
        item["vitals"] = torch.tensor(
            self.f["vitals"][self.f["hourly_index"] == j]
        ).float()
        item["survival"] = torch.tensor(
            self.f["survival"][self.f["hourly_index"] == j]
        )
        return item

    def pad_bincount(self, records):
        # get counts of each cod
        records = np.bincount(records)
        # pad each vector to length T, all possible codes
        padded = np.zeros(self.n_codes)
        padded[: len(records)] = records
        return torch.from_numpy(padded).float()

In [ ]:
def padded_collate(batch, pad_index, causal=False):
    res = {}
    treatment = torch.tensor(np.array([d["treatment"] for d in batch]))
    demog = torch.tensor(np.array([d["demog"] for d in batch])).float()
    if causal:
        res["treatment"] = torch.tensor(np.array([d["treatment"] for d in batch]))
        res["static"] = torch.tensor(np.array([d["demog"] for d in batch])).float()
    else:
        res["static"] = torch.cat([demog, treatment.unsqueeze(1)], 1)
    res["codes"] = torch.stack([d["codes"] for d in batch])
    res["vitals"] = pad_sequence(
        [d["vitals"] for d in batch], batch_first=True, padding_value=pad_index
    )
    res["survival"] = pad_sequence(
        [d["survival"] for d in batch], batch_first=True, padding_value=pad_index
    )
    return res

### Dynamic Survival Transformers Model Class

#### First some housekeeping

In [ ]:
# Import the os module
import os

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Install PyTorch Lightning
%pip install pytorch-lightning

# Install modules and libraries
import pytorch_lightning as pl
import torch
import math
from torch.nn import Linear
from src.metric import MeanAbsoluteError, ConcordanceIndex

#### DST Model Class

In [5]:
class DST(pl.LightningModule):
    def __init__(
        self,
        n_codes,
        n_vitals,
        n_demog,
        d_model,
        n_blocks,
        n_heads,
        dropout,
        pad,
        dynamic,
        causal,
        lr=0.001,
        alpha=0.01,
    ):
        super().__init__()
        self.save_hyperparameters()
        self.embed_codes = Linear(n_codes, d_model)
        d1 = 0 if causal else 1
        self.embed_static = Linear(d_model + n_demog + d1, d_model)
        self.embed_vitals = Linear(n_vitals, d_model)
        self.pos_encode = PositionalEncoding(d_model)
        self.pad = pad
        encoder_layer = torch.nn.TransformerEncoderLayer(
            d_model = d_model,
            nhead=n_heads,
            dropout=dropout,
            batch_first=True,
            dim_feedforward=d_model*4
        )
        norm = torch.nn.LayerNorm(d_model)
        self.transformer = torch.nn.TransformerEncoder(encoder_layer, n_blocks, norm)
        d2 = 1 if causal else 0
        self.to_hazard_c = torch.nn.Sequential(
            Linear(d_model + d2, d_model//2),
            torch.nn.ReLU(),
            Linear(d_model//2, 1),
            torch.nn.Sigmoid(),
        )
        self.train_mae = MeanAbsoluteError(pad=pad)
        self.val_mae = MeanAbsoluteError(pad=pad)
        self.val_ci = ConcordanceIndex(pad=pad)
        self.test_mae = MeanAbsoluteError(pad=pad)
        self.test_ci = ConcordanceIndex(pad=pad)

        # how much to weigh MAE loss
        self.alpha = alpha
        self.dynamic = dynamic
        self.causal = causal


    def forward(self, batch):
        # static features
        x = self.embed_codes(batch["codes"]).unsqueeze(1)
        x = self.embed_static(
            torch.cat([x, batch["static"].unsqueeze(1)], 2)
        )
        s = batch["vitals"].shape[1]
        # time-varying features
        if self.dynamic:
            pad_mask = (batch["vitals"][:, :, 0] == self.pad)
            x = x + self.embed_vitals(batch["vitals"])
            # autoregressive mask
            mask = (1 - torch.tril(torch.ones(s, s))).bool().cuda()

        else:
            mask = None
            x = x.repeat(1, s, 1)
            pad_mask = (batch["vitals"][:, :, 0] == self.pad)
        x = self.pos_encode(x)
        x = self.transformer(x, mask, pad_mask)
        if self.causal:
            t = torch.reshape(batch["treatment"], (-1, 1, 1))
            t = t.repeat(1, s, 1)
            # concatenate treatment as a new feature
            x = torch.cat((x, t), 2).float()
        # complement of hazard
        q_hat = self.to_hazard_c(x).squeeze(2)
        s_hat = q_hat.cumprod(1).clamp(min=1e-8)
        return s_hat

    def training_step(self, batch, batch_idx):
        s_hat =  self(batch)
        loss = self.combined_loss(s_hat, batch["survival"])
        self.log("train_loss", loss)
        self.train_mae(s_hat, batch["survival"])
        self.log("train_mae", self.train_mae, on_step=True, on_epoch=False)
        return loss



    def validation_step(self, batch, batch_idx):
        s_hat =  self(batch)
        loss = self.combined_loss(s_hat, batch["survival"])
        self.val_mae.update(s_hat, batch["survival"])
        self.val_ci.update(s_hat, batch["survival"])
        self.log("val_loss", loss)
        self.log("val_mae", self.val_mae, on_step=True, on_epoch=True)
        self.log("val_ci", self.val_ci, on_step=True, on_epoch=True)
        return loss


    def test_step(self, batch, batch_idx):
        s_hat =  self(batch)
        loss = self.combined_loss(s_hat, batch["survival"])
        self.test_mae.update(s_hat, batch["survival"])
        self.test_ci.update(s_hat, batch["survival"])
        self.log("test_mae", self.test_mae, on_step=True, on_epoch=True)
        self.log("test_ci", self.test_ci, on_step=True, on_epoch=True)
        return loss

    def predict_step(self, batch, batch_idx):
        # returns estimated survival times
        s_hat = self(batch)
        mask = (batch["survival"] != self.pad)
        return (s_hat * mask).sum(1)


    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)

    def ordinal_survival_loss(self, s_hat, y):
        # modified cross entropy loss
        nlog_survival = -torch.log(s_hat)
        nlog_failure = -torch.log(1 - s_hat)
        loss = 0
        loss += nlog_survival * torch.where(y==self.pad, 0, y)
        loss += nlog_failure * torch.where(y==self.pad, 0, (1-y))
        return loss.sum() / (y != self.pad).sum()
    
    def mae_loss(self, s_hat, y):
        observed = (y == 0).any(1).int()
        t_hat = torch.where(y == self.pad, 0, s_hat).sum(1)
        t = torch.where(y == self.pad, 0, y).sum(1)
        zeros = torch.zeros(t.shape).cuda()
        observed_error = torch.abs(t_hat - t) * observed
        censored_error = torch.maximum(zeros, t - t_hat) * (1 - observed)
        return (observed_error.sum() + censored_error.sum()) / t.numel()

    def combined_loss(self, s_hat, y):
        a = self.alpha
        ordinal_loss = self.ordinal_survival_loss(s_hat, y)
        mae_loss = self.mae_loss(s_hat, y)
        return (1 - a) * ordinal_loss + a * mae_loss


#### Define the positional encoding class

In [6]:
class PositionalEncoding(torch.nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(1), :].unsqueeze(0)
        return self.dropout(x)

#### Set up Weights and Biases

In [10]:
%pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00


In [11]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# New Section

In [ ]:
# import torch
from torch.utils.data import DataLoader, random_split
# import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

# set the default parameters

# preprocess
preprocess_seed = 30

# train
train_frac = .7
val_frac = .15
accelerator = "gpu"
devices = 1
max_epochs = 5
batch_size = 32
train_seed = 0

causal = False

# model
_target_ = "src.model.DST"
d_model = 32
n_blocks = 3
n_heads = 8
dropout = .1
pad = -100
dynamic = True
lr = 0.001
alpha = 0.01

# This call to the Mimi3Dataset class relies on having already run the preprocessing pipeline. If you have not run the preprocessing notebook, this will not work.

dataset = Mimic3Dataset(work_dir=cwd, seed=preprocess_seed)

train_size = int(train_frac * len(dataset))
if train_frac + val_frac == 1.0:
    val_size = len(dataset) - train_size
    test_size = 0
else:
    val_size = int(val_frac * len(dataset))
    test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(
    dataset,
    (train_size, val_size, test_size),
    torch.Generator().manual_seed(train_seed)
)

###### new code

def collate(x):
    return padded_collate(x, pad_index=pad, causal=causal)
train_loader = DataLoader(
    train_set,
    collate_fn=collate,
    batch_size=batch_size,
    shuffle=True
    )
val_loader = DataLoader(
    val_set, collate_fn=collate, batch_size=batch_size
)
if test_size:
    test_loader = DataLoader(
        test_set, collate_fn=collate, batch_size=batch_size
    )

model = DST(n_codes=dataset.n_codes, n_vitals=dataset.n_vitals, n_demog=dataset.n_demog, d_model=d_model, n_blocks=n_blocks, n_heads=n_heads, dropout=dropout, pad=pad, dynamic=dynamic, lr=lr, alpha=alpha, causal=causal)

callbacks = [ModelCheckpoint(monitor="val_mae_epoch", mode="min")]
trainer = pl.Trainer(
    accelerator=accelerator,
    devices=devices,
    max_epochs=max_epochs,
    callbacks=callbacks,
)
trainer.fit(model, train_loader, val_loader)
if test_size:
    trainer.test(dataloaders=test_loader)